In [1]:
import numpy as np
import keras
from keras.utils import plot_model
from keras.models import Sequential,Model,load_model
from keras.layers import Dense,Conv2D,concatenate,AveragePooling2D,MaxPool2D,BatchNormalization,Dropout,add,ZeroPadding2D,Flatten
from keras.callbacks import ModelCheckpoint,LearningRateScheduler,ReduceLROnPlateau
from keras.initializers import glorot_uniform
from keras.layers import Activation,Input
from keras.regularizers import l2
import os
import cv2
import warnings
warnings.filterwarnings(action='ignore')

Using TensorFlow backend.


In [2]:
dot_img_file = os.path.join(os.getcwd(),"model.png")
callbacks = [ModelCheckpoint(filepath = 'model_wight_best.h5',save_best_only=True),
             ReduceLROnPlateau(min_lr = 1e-7)] 
path = "C:\\Users\\Asus\\Desktop\\Face images\\model_wight_best.h5"

In [3]:
class ResNet_50:
    
    
    def __init__(self,class_,shape):
        self.CLASS = class_
        self.shape = shape
    
    
    def identity_block(self,inputs,f,filters):
            
        x = Conv2D(filters[0],kernel_size=1,strides=(1, 1),padding = 'valid',kernel_initializer=glorot_uniform(seed=0))(inputs)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        
        x = Conv2D(filters[1],kernel_size=(f,f),strides=(1, 1),padding = 'same',kernel_initializer=glorot_uniform(seed=0))(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
    
        x = Conv2D(filters[2],kernel_size=1,strides=(1, 1),padding = 'valid',kernel_initializer=glorot_uniform(seed=0))(x)
        x = BatchNormalization()(x)
        
        x = add([x,inputs])
        x = Activation('relu')(x)
        
        return x
        
    def convolutional_block(self,inputs,f,filters):
        
        x = Conv2D(filters[0],kernel_size=1,strides=(2, 2),padding = 'valid',kernel_initializer=glorot_uniform(seed=0))(inputs)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        
        x = Conv2D(filters[1],kernel_size=(f,f),strides=(1, 1),padding = 'same',kernel_initializer=glorot_uniform(seed=0))(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
    
        x = Conv2D(filters[2],kernel_size=1,strides=(1, 1),padding = 'valid',kernel_initializer=glorot_uniform(seed=0))(x)
        x = BatchNormalization()(x)
        
        inputs = Conv2D(filters[2],kernel_size=1,strides=(2, 2),padding = 'valid',kernel_initializer=glorot_uniform(seed=0))(inputs)
        inputs = BatchNormalization()(x)
        
        x = add([x,inputs])
        x = Activation('relu')(x)
        return x
        
        
    def forward(self):
        inputs = Input(shape = self.shape)
        x = Conv2D(64, (7, 7), strides=(2, 2),kernel_initializer=glorot_uniform(seed=0))(inputs)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = MaxPool2D((3, 3), strides=(2, 2))(x)
        
        x = self.convolutional_block(x, f = 3, filters = [128, 128, 512])
        x = self.identity_block(x, 3, [128, 128, 512])
        x = self.identity_block(x, 3, [128, 128, 512])
        x = self.identity_block(x, 3, [128, 128, 512])

        x = self.convolutional_block(x, f = 3, filters = [256, 256, 1024])
        x = self.identity_block(x, 3, [256, 256, 1024])
        x = self.identity_block(x, 3, [256, 256, 1024])
        x = self.identity_block(x, 3, [256, 256, 1024])
        x = self.identity_block(x, 3, [256, 256, 1024])
        x = self.identity_block(x, 3, [256, 256, 1024])

        x = self.convolutional_block(x, f = 3, filters = [512, 512, 2048])
        x = self.identity_block(x, 3, [512, 512, 2048])
        x = self.identity_block(x, 3, [512, 512, 2048])

        x = AveragePooling2D((2,2), name="avg_pool")(x)
        x = Flatten()(x)
        x = Dense(self.CLASS, activation='softmax',kernel_initializer = glorot_uniform(seed=0))(x)

        model = Model(inputs = inputs, outputs = x, name='ResNet50')
        return model
        

In [4]:
resnet = ResNet_50(5,(64,64,3))
model = resnet.forward()

In [5]:
#plot_model(model,dot_img_file)#

In [6]:
model = load_model(path)

In [7]:
mapp = {}
path2 = "C:\\Users\\Asus\\Desktop\\Files\\Machine learning,Deep Learning\\Emotion detection images\\images\\train"
for i,maps in enumerate(os.listdir(path2)):
    mapp[i] = maps

In [9]:
cascades_path = os.path.join("C:\\Users\\Asus\\Desktop\\Files",'haarcascade_frontalface_alt2.xml')

In [ ]:
cap = cv2.VideoCapture(0)
font                   = cv2.FONT_HERSHEY_SIMPLEX
bottomLeftCornerOfText = (10,150)
fontScale              = 1
fontColor              = (255,255,255)
lineType               = 2

while True : 
    
    ret,frame = cap.read()
    image = frame.copy()
    if ret == False :
        break
    (h, w) = image.shape[:2]
    detect = cv2.CascadeClassifier(cascades_path)
    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    faces = detect.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in faces:
        startx,starty,endx,endy = (x,y,x+w,y+h)
        frame = cv2.rectangle(frame,(startx,starty),(endx,endy),[0,255,0],2)
        face = image[starty:endy, startx:endx]    
        face = cv2.resize(face,(300,300))
        inp = np.expand_dims(np.array(face),axis = 0)
        pred = model.predict(inp)
        cv2.putText(frame,mapp[np.argmax(pred)], bottomLeftCornerOfText, font, fontScale,fontColor,lineType)
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()